# LU-Zerlegung: Vor- / Rückwärtseinsetzen

## Ziel

In dieser Lektion implementieren wir das Lösungsverfahren $A x = b$ unter der Annahme, dass eine $LR$-Zerlegung $A = L R$ der Matrix $A$ bereits gegeben ist.
 
 **Abgabe**: Sie geben als Praktikumsbericht ihren Code in Matlab oder Python (oder C, oder ...) ab. Für die Bewertung ist ausschlaggebend, dass Sie sich ersichtlich mit der Aufgabe auseinandergesetzt haben, unabhängig davon, ob der Code am Ende läuft oder nicht. Wir werden allerdings im nachfolgenden Praktikum darauf aufbauen.

## Theorie

Lineare Gleichungssysteme $Ax = b$ für $A \in \mathbb{R}^{n \times n}, b \in \mathbb{R}^n$ können durch Vorwärts- bzw. Rückwärtseinsetzen gelöst werden, wenn die Matrix $A=R$ eine obere bzw. $A=L$ eine untere Dreiecksmatrix ist. Sie implementieren und testen in diesem Praktikum die beiden Algorithmen. Das *Vorwärtseinsetzen* ist Algorithmus 1 im Skript. Rückwärtseinsetzen funktioniert analog.

Je nach Kontext kann zudem eine effiziente Implementierung entscheidend sein, sowohl memory-efficient (kein unnötiges Belegen von Speicherplatz) wie auch time-efficient. Im einzelnen bedeutet das

 * Die LU-Zerlegung wird in-place ausgeführt, wobei die ursprüngliche Matrix $A$ überschrieben wird. Die resultierende Matrix LR enthält im Dreieck links unten die Matrix $L$, ohne die Diagonalelemente und im Dreieck oben rechts die Matrix R (inklusive Diagonale). Dies enspricht der Vorgehensweise im Unterricht.
 * Es werden keine Zeilenvertauschungen explizit durchgeführt, stattdessen wird mit einem Indexvektor gearbeitet. 
 


## Aufgaben

### Aufgabe 1 

1. Beschreiben Sie das Vorwärtseinsetzen als Pseudocode, analog zu Algorithmus $1$ im Skript.
2. Beschreiben Sie in Pseudocode das Vor- und Rückwärtseinsetzen, wenn die LU-Zerlegung in *einer* Matrix $LR$ gegeben ist. Beachten Sie, dass der Vektor $x$ beim Rückwärtseinsetzen überschrieben werden kann. Drücken Sie die Operationen soweit möglich als Skalarprodukte aus.
3. Beschreiben Sie die in-place LU-Zerlegung ohne Zeilentausch als Pseudocode. 

**1. Pseudocode Vorwärtseinsetzen:** 

*Eingabe:* Eine obere Dreiecksmatrix A und ein Vektor b

n = Anzahl der Zeilen in der Matrix A

x = Ein n-dimensionaler Nullvektor

// Beginne mit der letzten Zeile der Matrix A

for i = n-1 down to 0:

    sum = 0
    
    // Summiere die bereits bekannten x-Werte und die Werte in A für diese Zeile
    
    for j = i+1 to n-1:
    
        sum += A[i,j] * x[j]
        
    // Berechne das neue x für diese Zeile
    
    x[i] = (b[i] - sum) / A[i,i]

// Gebe den Vektor x zurück

return x

*Ausgabe:* Eine Lösungsmenge x für das Gleichungssystem Ax = b



**2. Vorwärtseinsetzen:**

Für j = n, n − 1, . . . , 1 berechne
$$y_j = \frac{ b_j - \sum\limits^{j-1}_{k=1} l_{j,k} y_k}{ l_{j,j} }$$

**Rückwärtseinsetzen:**

Für j = n, n − 1, . . . , 1 berechne
$$x_j = \frac{ y_j - \sum\limits^{n}_{k=j+1} r_{j,k} x_k}{ r_{j,j} }$$

**3. in-place LU-Zerlegung ohne Zeilenvertauschen**

*Eingabe:* Matrix A

     // n-1 Iterationsschritte
     
   for i := 1 to n-1
   
     // Zeilen der Restmatrix werden durchlaufen 
     
     for k := i+1 to n
     
       // Berechnung von L
       
       A(k,i) := A(k,i) / A(i,i) // Achtung: vorher Prüfung auf Nullwerte notwendig
       
       // Spalten der Restmatrix werden durchlaufen
       
       for j := i+1 to n
       
         // Berechnung von R
         
         A(k,j) := A(k,j) - A(k,i) * A(i,j)
         
 *Ausgabe:* Matrix A (in modifizierter Form)

### Aufgabe 2

Implementieren Sie eine Funktion **fbSubs** für das Vorwärts- und Rückwärtseinsetzen. Versuchen Sie, so wenig Schleifen wie möglich zu verwenden!

In [1]:
import numpy as np

"""
forwardSubs: Vorwärtseinsetzen
in: 
 - Matrix LR, die das Ergebnis einer LU-Zerlegung enthält
 - Vektor b: rechte Seite des LGS
out: Lösung x des LGS
"""  

#LRtest = np.array([[1,2,3],[1,-1,-2],[3,3,-2]])
#btest = np.array([1,-1,2])

def forwardSubs(LR, bIn):
    b=bIn.copy()
    n = len(b)
    y = b
    for i in range(1,n):
        y[i] = (b[i] - np.sum(LR[i,:i]*y[:i]))
    return y


#ytest = forwardSubs(LRtest,btest)

#print(ytest)

"""
backwardSubs: Rückwärtseinsetzen
in: 
 - Matrix LR, die das Ergebnis einer LU-Zerlegung enthält
 - Vektor b: rechte Seite des LGS
out: Lösung x des LGS
"""  

#LRtest = np.array([[1,2,3],[1,-1,-2],[3,3,-2]])
#ytest = np.array([1,-2,5])

def backwardSubs(LR, y):
    n = len(y)
    x = y/np.diagonal(LR)
    for i in range(n-1, -1, -1):
        x[i] = (y[i] - np.sum(LR[i,(i+1):]*x[(i+1):]))/LR[i,i]
    return x

#xtest = backwardSubs(LRtest,ytest)

#print(xtest)

"""
fbSubs: Löst das Gleichungssystem LR*x=b
in: 
 - Matrix LR, die das Ergebnis einer LU-Zerlegung enthält
 - Vektor b: rechte Seite des LGS
out: Lösung x des LGS
"""  
def fbSubs(LR, b):
    y = forwardSubs(LR, b)
    x = backwardSubs(LR, y)
    return x

#LRtest = np.array([[1,2,3],[1,-1,-2],[3,3,-2]])
#btest = np.array([1,-1,2])
#L,R = np.tril(LRtest,-1)+np.identity(3), np.triu(LRtest) 
#res = L@R@fbSubs(LRtest,btest)
#print(res-btest)




### Aufgabe 3

Testen Sie **fbSubs** zuerst mit dem Zahlenbeispiel aus dem Unterricht (wo Sie alles schrittweise nachvollziehen können) und dann mit dem folgenden Testcode, den Sie auch variieren dürfen, bis Sie sicher sind, dass alles wie gewünscht funktioniert. Sie müssen sich im nachfolgenden Praktikum auf **fbSubs** verlassen können!

In [2]:
#Test LR
n = 7 # Dimension der Koeffizientenmatrix
for k in range(1000):                      # Testläufe
    LR = np.array(np.random.rand(n,n))   # zufällige Matrix LR 
    rhs = np.array(np.random.rand(n))    # zufällige rechte Seite des LGS
    x = fbSubs(LR, rhs)   # Aufruf der eigenen Funktion
    
    L,R = np.tril(LR,-1)+np.identity(n), np.triu(LR) # L und R extrahieren
    
    assert(np.linalg.norm(rhs - L@R@x) < 1e-10)     # Test, mit numerischer Toleranz

# Implementierung Gauss-Algorithmus mit Spaltenpivotisierung

## Ziel

 In diesem Praktikum implementieren wir das Lösungsverfahren $A x = b$ inklusive $LU$-Zerlegung $P A = L R$ der Matrix $A$ mit Spaltenpivotisierung.
 
 **Abgabe**: Sie geben als Praktikumsbericht ihren Code in Matlab oder Python (oder C, oder ...) ab. Für die Bewertung ist ausschlaggebend, dass Sie sich ersichtlich mit der Aufgabe auseinandergesetzt haben, unabhängig davon, ob der Code am Ende läuft oder nicht. Sie können den Code direkt in das notebook schreiben oder in einer Entwicklungsumgebung Ihrer Wahl. 

## Theorie

Ein lineares Gleichungssystem $Ax = b$ kann gelöst werden, indem die Matrix $A$ in eine untere bzw. obere Dreiecksmatrix zerlegt wird: $A = L R$, falls keine Zeilenvertauschungen notwendig sind bzw. $P A = L R$ mit der zusätzlichen *Permutationsmatrix* $P$, falls Zeilen vertauscht werden. 

Diesen Prozess nennt man auf Deutsch $LR$-Zerlegung (englisch: $LU$-factorization, für *lower, upper*). Der Algorithmus ist eine kleine Erweiterung des Gauss-Algorithmus (Algorithmus 2 im Skript). $R$ ist die gewohnte Zeilenstufenform, in $l_{ij}$ merken wir uns zusätzlich, mit welchem Faktor Zeile $i$ von Zeile $j$ subtrahiert wurde. Die Beschreibung des Vorgehens entnehmen Sie dem Unterricht.

Sind Zeilenvertauschungen notwendig (oder auf Grund einer Pivotstrategie erwünscht), so führt die Permutationsmatrix $P$ Buch über diese Vertauschungen: alle Zeilenvertauschungen in $A$ werden auch in der Matrix $P$ übernommen.

Je nach Kontext kann zudem eine effiziente Implementierung entscheidend sein, sowohl memory-efficient (kein unnötiges Belegen von Speicherplatz) wie auch time-efficient. Im einzelnen bedeutet das

 * Die LU-Zerlegung wird *in-place* ausgeführt, wobei die ursprüngliche Matrix $A$ überschrieben wird. Die resultierende Matrix LR enthält im Dreieck links unten die Matrix $L$, ohne die Diagonalelemente und im Dreieck oben rechts die Matrix R (inklusive Diagonale). Dies enspricht der Vorgehensweise im Unterricht.
 * Zeilenvertauschungen können formal durch eine Permutationsmatrix $P$ erfasst werden. Es gilt dann $L R = P A$ und man hat wegen $A x = b$ <=> $P A x = P b$ die beiden LGS 
 
     $L \cdot y = P \cdot b$    und    $R \cdot x = y$

zu lösen. Um die Matrixmultiplikation mit der Permutationsmatrix zu sparen  arbeitet man stattdessen mit einem *Indexvektor* $i$, der zu Beginn auf $i = [0, 1, ..., n-1]$ initialisiert wird. Werden Zeilen in $A$ getauscht, so tauscht man auch die entsprechenden Einträge in $i$ und löst am Ende die beiden LGS

$L \cdot y = b[i]$     und    $R \cdot x = y$

Ziel dieses Praktikums ist eine Funktion **linsolve**, die LGS $A \cdot x = b$ mit Hilfe einer *in-place* *LU-Zerlegung* mit Spaltenpivotisierung löst. Wir verwenden die Ergebnisse des letzten Prakikums:

## Aufgaben

### Aufgabe 1

Implementieren Sie die *LU-Zerlegung*: die Funktion **LU** nimmt als Input eine quadratische Matrix $A$ und gibt Dreiecksmatrizen $L, R$ zurück, so dass $L R = A$ gilt. Im ersten Schritt lassen wir die Pivotisierung weg. Der Indexvektor **idx** bleibt dann unverändert.

In [3]:
# LU-Zerlegung der quadratischen Matrix A    
# in: quadratische Matrix A
#out: 
# - A wird überschrieben, unteres Dreieck = L (ohne Diagonale), oberes Dreieck = R
# - idx: Indexvektor der Zeilenvertauschungen

import numpy as np

def LU(A):
    m = A.shape[0]
    idx = np.array(range(m))
    for i in range(m):                  #jedes Pivotelement durchgehen
        max = np.argmax(np.abs(A[i:,i])) + i
        A[[i,max]] = A[[max,i]]
        idx[[i,max]]= idx[[max,i]]
        for j in range(i+1,m):          #alle Zeilen unterhalb des jeweiligen Pivotelements durchgehen
            fac = -(A[j,i] / A[i,i])    #Faktor für die aktuelle Zeile ausrechnen
            A[j,i:] += fac * A[i,i:]      #Jeweilige Zeile mit der ersten Zeile mal dem Faktor addieren
            A[j,i] = -fac               #Den Faktor direkt in die untere Matrix speichern
    return A, idx

A = np.array([[3.,6,3],[1,3,6],[6,3,3]])
LU(A)


(array([[6.        , 3.        , 3.        ],
        [0.5       , 4.5       , 1.5       ],
        [0.16666667, 0.55555556, 4.66666667]]),
 array([2, 0, 1]))

Testen Sie **LU** mit zufällig erzeugten Matrizen.  

In [4]:
n = 3
#test LU
for k in range(1000):
    A = np.array( np.random.rand(n,n) )   # zufällige Matrix A erzeugen
    LR, idx = LU(A.copy())                  # LU-Zerlegung von A   
    L,R = np.tril(LR,-1)+np.identity(n), np.triu(LR)  # Matrizen L, R extrahieren  
    assert( np.linalg.norm(L@R - A[idx]) < 1e-8)

print("test completed")

test completed


### Aufgabe 2

Erstellen Sie eine Funktion **linsolve(A, b)**, die für eine Matrix $A$ und einen Vektor $b$ das LGS $A x = b$ mit Hilfe der *LR-Zerlegung* und dem Vorwärts- und Rückwärtseinsetzen löst.

In [9]:
# lineares Gleichungssystem A*x = b lösen.
def linsolve(A, b):
    LR, idx = LU(A.copy())
    print(idx)
    print(LR)
    print(LR[idx])
    x = fbSubs(LR[idx],b)
    return x

A = np.array([[1.,3,-2],[2,-1,0],[-1,1,3]])
b = np.array([3.,1,13])

res = linsolve(A,b)
print(res)

[1 0 2]
[[ 2.         -1.          0.        ]
 [ 0.5         3.5        -2.        ]
 [-0.5         0.14285714  3.28571429]]
[[ 0.5         3.5        -2.        ]
 [ 2.         -1.          0.        ]
 [-0.5         0.14285714  3.28571429]]
[-10.47826087   5.           4.63043478]


Testen Sie **linsolve** mit zufällig erzeugten Matrizen

In [10]:
#test linsolve
for k in range(10):
    A = np.random.rand(n,n)
    rhs = np.random.rand(n)
    x = linsolve(A, rhs)   
    assert(np.linalg.norm(rhs - A @ x) < 1e-8)

[2 0 1]
[[ 0.51594279  0.30287398  0.28973979]
 [ 0.97600906  0.33775271  0.68357463]
 [ 0.1652526   0.50349335 -0.37612616]]
[[ 0.1652526   0.50349335 -0.37612616]
 [ 0.51594279  0.30287398  0.28973979]
 [ 0.97600906  0.33775271  0.68357463]]


AssertionError: 

Mit der folgenden Funktion *rndCond(n, cond)* können Sie zufällige $n \times n$ Matrizen mit vorgegebener Konditionszahl *cond* erzeugen. 

In [ ]:
import numpy.linalg as lin
import numpy.random as rnd

# random orthogonal matrix
def rndOrtho(n):
    S = rnd.rand(n,n)
    S = S - S.T
    O = lin.solve(S - np.identity(n), S + np.identity(n))
    return O

# random matrix with specified condition number
def rndCond(n, cond):    
    d = np.logspace(-np.log10(cond)/2, np.log10(cond)/2,n);
    A = np.diag(d)
    U,V = rndOrtho(n), rndOrtho(n)
    return U@A@V.T

Verwenden Sie diese, um Ihre Implementation von *linsolve* zu testen. Berechnen Sie den relativen Fehler $\frac{|\Delta_x|}{|x|}$ und vergleichen Sie mit dem Ergebnis von *linsolve* aus der Bibliothek *numpy.linalg*. 

In [ ]:
N = 1
for k in range(N):
    A = rndCond(n, 1e14)
    b = rnd.rand(n)
    x = linsolve(A,b)
    dx = (b - x@A)
    xrel = 0
    print(np.linalg.norm(dx))

    

2031418483295.4785


### Aufgabe 3

Implementieren Sie die Spaltenpivotisierung und wiederholen Sie die Testläufe aus Aufgabe 2

In [ ]:
# Die LU-Zerlegung mit Pivotisierung funktioniert so, allerdings funktioniert nun die linsolve FUnktion nicht mehr, 
#wir haben es noch nicht geschafft die linsolve mit Pivotisierung zum laufen zu bringen 

import numpy as np

def LU(A):
    m = A.shape[0]
    idx = np.array(range(m))
    for i in range(m):                  #jedes Pivotelement durchgehen
        max = np.argmax(np.abs(A[i:,i])) + i
        A[[i,max]] = A[[max,i]]
        idx[[i,max]]= idx[[max,i]]
        for j in range(i+1,m):          #alle Zeilen unterhalb des jeweiligen Pivotelements durchgehen
            fac = -(A[j,i] / A[i,i])    #Faktor für die aktuelle Zeile ausrechnen
            A[j,i:] += fac * A[i,i:]      #Jeweilige Zeile mit der ersten Zeile mal dem Faktor addieren
            A[j,i] = -fac               #Den Faktor direkt in die untere Matrix speichern
    return A, idx

A = np.array([[3.,6,3],[1,3,6],[6,3,3]])
LU(A)
